In [1]:
import pandas as pd
from ddr import *
pd.options.mode.copy_on_write = True

In [2]:
# kolommen controleren controleren met df.columns zonder 'usecols='
vl = pd.read_csv('vrlijst_lijstdef.csv',
                  usecols=[0, 1, 2, 5, 8, 12, 16, 19],
                  dtype={'VERVALLEN': 'boolean', 'DATUM': 'string'})

In [13]:
vl.loc[vl['SOORTID'] == 'C']

In [4]:
vr = pd.read_csv('vrlijst_vragen.csv', low_memory=False,
                usecols=[0, 1, 3, 6, 7, 8, 12, 27, 33, 39, 40],
                dtype={'MAGLEEG': 'boolean', 'VERVALLEN': 'boolean', 'ENKELV': 'boolean',
                       'CONTROLID': 'Int64'})

In [5]:
vc = pd.read_csv('vrlijst_controls.csv',
                usecols=[0, 1, 4])

In [ ]:
def vraag_type(vraag):
    descr = vc.loc[vc['CONTROLID'] == vraag.CONTROLID].to_dict('records')
    return descr[0]['NAAM']

In [6]:
tl = pd.read_csv('vrlijst_treelay.csv',
                 usecols=[0, 1, 2, 3, 4, 8, 9, 12, 13],
                 dtype={'VOLGNR':'Int64', 'HERHAALBAA': 'boolean', 'ONZZANTW': 'boolean', 'NEWPAGE': 'boolean'})

In [7]:
start = Lijst.make_one(vl.loc[vl['LIJSTID'] == 'CS00065113'])

In [9]:
def add_children(vlnode, distance=0):
    tlnodes = tl.loc[tl['LIJSTID'] == vlnode.LIJSTID].copy()
    tlnodes.sort_values(by='VOLGNR', inplace=True)
    for node in tlnodes.to_dict('records'):
        childid = node['CHILDID']
        indent = '  ' * distance
        onz = node['ONZZANTW']
        if node['TYPE'] == 'V':
            vraag = Vraag.make_one(vr.loc[vr['VRAAGID'] == childid])
            vlnode.children.append(vraag)
            print(f"{indent}{node['VOLGNR']} V: {vraag} type={vraag_type(vraag)} verberg={onz}")
        elif node['TYPE'] == 'L':
            lijst = Lijst.make_one(vl.loc[vl['LIJSTID'] == childid])
            add_children(lijst, distance+1)
            vlnode.children.append(lijst)
            print(f"{indent}{node['VOLGNR']} L: {lijst} verberg={onz}")
        else:
            print(f"{indent}onbekend type {node['TYPE']}")

In [10]:
add_children(start)